#### Identifying bird species infected with H5N1 and finding bird density from past 30 days in Chicago (as of 7/15/2025).   
By: Angel Moreno

In [32]:
import pandas as pd
import geopandas as gpd
import os
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import numpy as np
from shapely import Point
from dotenv import load_dotenv

#### Bird species infected with H5N1 in Midwestern states of interest:

In [33]:
df = pd.read_csv("HPAI Detections in Wild Birds.csv")

mw_states_of_interest = ["Illinois", "Wisconsin", "Michigan"]
dead_birds = df[(df["HPAI Strain"].str.contains("H5N1")) & (df["State"].isin(mw_states_of_interest)) & (df["Sampling Method"]=="Morbidity/Mortality")]
dead_birds["Bird Species"].unique()

array(['Bald eagle', 'Peregrine falcon', 'Sandhill crane',
       'Red-tailed hawk', 'Herring gull', 'Red-breasted merganser',
       'Great horned owl', 'Hawk (unidentified)', 'American crow',
       'Canada goose', 'Trumpeter swan', 'Bufflehead', 'Snowy owl',
       'Tundra swan', 'Chilean flamingo', 'Barred owl', 'Mallard',
       'Hooded merganser', 'Common loon', 'European starling',
       'Snow goose', 'Canvasback', 'Crow (unidentified)', "Ross's goose",
       'American white pelican', 'Common tern', 'Swan (unidentified)',
       'Blue-winged teal', 'Goose (unidentified)', 'Great blue heron',
       'Great egret', 'Pelican (unidentified)', 'Ring-billed gull',
       'Caspian tern', 'Double-crested cormorant', 'Wood duck',
       'Redhead duck', 'Red-winged blackbird', 'Gadwall',
       'Turkey vulture', 'Common raven', 'Ruddy duck',
       'Eagle (unidentified)', 'Rough-legged hawk', 'Red-shouldered hawk',
       "Cooper's hawk", 'Wood Duck', 'Mute swan',
       'Cormorant (uni

In [34]:
# Illinois only
dead_birds_il = df[(df["HPAI Strain"].str.contains("H5N1")) & (df["State"] == "Illinois") & (df["Sampling Method"]=="Morbidity/Mortality")]
dead_birds_il["Bird Species"].unique()

array(['Chilean flamingo', 'Red-breasted merganser', 'Bufflehead',
       'Canada goose', 'Mallard', 'Hooded merganser', 'Sandhill crane',
       'Snow goose', 'Crow (unidentified)', 'Bald eagle',
       'Red-tailed hawk', 'Great horned owl', "Ross's goose",
       'American white pelican', 'Common tern', 'Goose (unidentified)',
       'Swan (unidentified)', 'Pelican (unidentified)',
       'Double-crested cormorant', 'Turkey vulture',
       'Cormorant (unidentified)'], dtype=object)

#### Chicago Bird Migration 30-Day Data (eBird):

In [41]:
load_dotenv() 

api_key = os.getenv("EBIRD_API_KEY")
headers = {"X-eBirdApiToken": api_key}

params = {
    "lat": 41.8781,
    "lng": -87.6298,
    "dist": 10, # 10km radius (will filter with shp file later)
    "back": 1, # past 30 days
    "maxResults": 10000
}

response = requests.get(
    "https://api.ebird.org/v2/data/obs/geo/recent",
    headers=headers,
    params=params
)



In [42]:
data = response.json()

In [43]:
obs_30d = pd.DataFrame(data)
obs_30d.shape

(32, 14)

In [ ]:
# chicago_boundary = gpd.read_file("Chicago_Tracts_2020.zip")

# # Make sure both GeoDataFrames use the same CRS
# chicago_boundary = chicago_boundary.to_crs("EPSG:4326")

# # Convert eBird DataFrame to GeoDataFrame
# geometry = [Point(xy) for xy in zip(obs_30d['lng'], obs_30d['lat'])]
# gdf = gpd.GeoDataFrame(obs_30d, geometry=geometry, crs="EPSG:4326")

# # Use unary_union to create a single polygon from tracts
# chicago_union = chicago_boundary.union_all()

# # Filter: keep only points within the city boundary
# gdf_chicago = gdf[gdf.within(chicago_union)]

# gdf_chicago

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,geometry
0,cangoo,Canada Goose,Branta canadensis,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-07-17 06:42,56.0,41.963383,-87.634420,True,False,False,S260508009,NaN,POINT (-87.63442 41.96338)
1,chiswi,Chimney Swift,Chaetura pelagica,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-07-17 06:42,1.0,41.963383,-87.634420,True,False,False,S260508009,NaN,POINT (-87.63442 41.96338)
2,amhgul1,American Herring Gull,Larus smithsonianus,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-07-17 06:42,2.0,41.963383,-87.634420,True,False,False,S260508009,NaN,POINT (-87.63442 41.96338)
3,barswa,Barn Swallow,Hirundo rustica,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-07-17 06:42,2.0,41.963383,-87.634420,True,False,False,S260508009,NaN,POINT (-87.63442 41.96338)
4,eursta,European Starling,Sturnus vulgaris,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-07-17 06:42,22.0,41.963383,-87.634420,True,False,False,S260508009,N,POINT (-87.63442 41.96338)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,trokin,Tropical Kingbird,Tyrannus melancholicus,L108460,"Douglass (Anna & Frederick) Park, Chicago",2025-06-30,1.0,41.863321,-87.699156,True,True,False,S256637333,NaN,POINT (-87.69916 41.86332)
183,laugul,Laughing Gull,Leucophaeus atricilla,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-06-28 18:26,1.0,41.963383,-87.634420,True,True,False,S255042055,NaN,POINT (-87.63442 41.96338)
188,comloo,Common Loon,Gavia immer,L383782,"Rainbow Beach, Chicago",2025-06-24 15:30,1.0,41.758805,-87.546078,True,True,False,S253794481,NaN,POINT (-87.54608 41.75881)
189,prawar,Prairie Warbler,Setophaga discolor,L161180,"Montrose Point Bird Sanctuary, Lincoln Park, C...",2025-06-23 20:14,1.0,41.963383,-87.634420,True,True,False,S253572510,NaN,POINT (-87.63442 41.96338)


In [23]:
print(type(gdf_chicago))
print(type(chicago_boundary))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [24]:
projected_crs = "EPSG:4326"
chicago_boundary = chicago_boundary.to_crs(projected_crs)
gdf_chicago = gdf_chicago.to_crs(projected_crs)

date_format = "%Y-%m-%d %H:%M"

gdf_chicago["date"] = pd.to_datetime(gdf_chicago["obsDt"], format='ISO8601')

# will be used for animation
unique_dates = np.unique(gdf_chicago['date'])
len(unique_dates)

42

In [26]:
for day in unique_dates:
    fig, ax = plt.subplots(figsize=(8, 8))
    chicago_boundary.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.1, zorder=1)

    plt.grid(color='gray', linestyle='--', linewidth=0.5)
    
    # day = day.reset_index(drop=True) # to prevent keyerror
    title = f"Bird Activity on {str(day)[0:10]}"
    ax.set_title(title, fontsize=14)

    var_to_plot = "howMany"

    gdf_chicago[gdf_chicago["date"]==day].plot(
        column=var_to_plot,
        cmap="RdPu",
        legend=True,
        ax=ax,
        edgecolor="black",
        linewidth=0.2
    )

    folder_name = os.path.join("weather_var_maps")
    os.makedirs(folder_name, exist_ok=True)
    path = os.path.join(folder_name, f"{title}.png")

    plt.savefig(path)
    plt.close(fig)

In [27]:
images = []
frame_files = sorted(os.listdir("weather_var_maps"))
for file in frame_files:
    if file.endswith(".png"):
        images.append(imageio.imread(os.path.join("weather_var_maps", file)))

imageio.mimsave("bird_activity_heatmap.gif", images, fps=2)

In [ ]:
# meeting 7/17/25:
# use chicago boundary and not census tract specific boundaries
# 10km outside chicago is fine (for density)
# Getis-Ord Gi* -> look into this method (for hotpots hot and cool spots)